In [17]:
import funciones_creditos as fc
import numpy as np
import numpy_financial as npf
import pandas as pd
from IPython.display import display
from sqlalchemy import create_engine

# ---------------------------
# Parámetros de conexión a la base de datos
# ---------------------------
DATABASE_TYPE = 'postgresql'
USER = 'ricardo.torres'
PASSWORD = r'UW`bv&&rg>2!kwo\eOaD'
HOST = '34.123.217.240'
PORT = '5432'
DATABASE = 'credito'

database_uri = f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(database_uri)

# ---------------------------
# Consulta SQL
# ---------------------------
query = """
SELECT
    credits.id AS id_credito,
    credits.deposit_reference AS referencia_deposito,
    credits.amount AS saldo_inicial,
    credits.annual_interest_rate AS tasa_interes_anual,
    credits.payment_amount AS cuota_mensual,
    credits.term AS plazo,
    credits.operational_opening_date AS fecha_apertura,
    credits.first_payment_date AS fecha_primer_pago,
    CASE 
        WHEN NOT credits.open THEN LEAST(
            (SELECT MAX(payments_2.date) FROM payments AS payments_2 WHERE payments_2.credit_id = credits.id),
            credits.closing_date
        )
        ELSE NULL
    END AS fecha_cierre,
    payments.date AS fecha_pago,
    payments.amount AS monto_pago
FROM credits
    LEFT JOIN payments ON credits.id = payments.credit_id
WHERE
    credits.country = 'mx'
    AND credits.product_id IN (7, 8, 9, 10, 11)
ORDER BY credits.id, payments.date;
"""

df_creditos_pagos = pd.read_sql_query(query, engine)
engine.dispose()

# ---------------------------
# Conversión de fechas y obtención del último registro de cada crédito
# ---------------------------
fecha_columnas = ["fecha_apertura", "fecha_pago", "fecha_primer_pago", "fecha_cierre"]
df_creditos_pagos[fecha_columnas] = df_creditos_pagos[fecha_columnas].apply(pd.to_datetime, errors='coerce')
df_creditos = df_creditos_pagos.groupby('id_credito').last()

# ---------------------------
# Constantes
# ---------------------------
IVA = 0.16  
FACTOR_IVA = 1 + IVA  
FECHA_CORTE = pd.Period("2021-12", freq='M')

# ---------------------------
# Preparación de datos de pagos
# ---------------------------
df_creditos_pagos = df_creditos_pagos.sort_values(by=['id_credito', 'fecha_pago'])
df_creditos_pagos['fecha_anterior'] = df_creditos_pagos.groupby('id_credito')['fecha_pago'].shift(1)
df_creditos_pagos['fecha_anterior'] = df_creditos_pagos['fecha_anterior'].fillna(df_creditos_pagos['fecha_apertura'])
df_creditos_pagos['dias_transcurridos'] = (df_creditos_pagos['fecha_pago'] - df_creditos_pagos['fecha_anterior']).dt.days
df_creditos_pagos['monto_pagado_acumulado'] = df_creditos_pagos.groupby('id_credito')['monto_pago'].cumsum()
df_creditos_pagos['tasa_mensual'] = (df_creditos_pagos['tasa_interes_anual'] / 100) * FACTOR_IVA / 12
df_creditos_pagos['num_pagos_realizados'] = df_creditos_pagos['monto_pagado_acumulado'] / df_creditos_pagos['cuota_mensual']

# ---------------------------
# Cálculo de Saldo Posterior con Valor Futuro
# ---------------------------
df_creditos_pagos['saldo_posterior_valor_futuro'] = npf.fv(
    rate=df_creditos_pagos['tasa_mensual'], 
    nper=df_creditos_pagos['num_pagos_realizados'], 
    pmt=df_creditos_pagos['cuota_mensual'], 
    pv=-df_creditos_pagos['saldo_inicial'], 
    when=0  
)
df_creditos_pagos['saldo_posterior_valor_futuro'] = df_creditos_pagos['saldo_posterior_valor_futuro'].clip(lower=0)

# Inicialización para el cálculo de saldo posterior por intereses acumulados
df_creditos_pagos['saldo_posterior_intereses_acumulados'] = np.nan

# ---------------------------
# Cálculo del saldo posterior de intereses acumulados
# ---------------------------
for id_credito, grupo in df_creditos_pagos.groupby('id_credito'):
    saldo_actual = df_creditos.loc[id_credito, 'saldo_inicial']
    tasa_interes_anual = df_creditos.loc[id_credito, 'tasa_interes_anual'] / 100
    tasa_interes_diaria = tasa_interes_anual / 360
    intereses_acumulados = 0
    saldo_posterior_intereses_acumulados = []
    
    for _, renglon in grupo.iterrows():
        interes_generado = max(renglon['dias_transcurridos'] * tasa_interes_diaria * saldo_actual, 0)
        intereses_acumulados += interes_generado
        
        if renglon['monto_pago'] < intereses_acumulados * FACTOR_IVA:
            intereses_acumulados -= renglon['monto_pago'] / FACTOR_IVA
        else:
            saldo_actual -= renglon['monto_pago'] - (intereses_acumulados * FACTOR_IVA)
            intereses_acumulados = 0
        
        saldo_posterior_intereses_acumulados.append(saldo_actual)
    
    df_creditos_pagos.loc[grupo.index, 'saldo_posterior_intereses_acumulados'] = saldo_posterior_intereses_acumulados

# ---------------------------
# Selección del saldo posterior según la fecha de corte
# ---------------------------
df_creditos_pagos['mes_apertura'] = df_creditos_pagos['fecha_apertura'].dt.to_period('M')
df_creditos_pagos['saldo_posterior'] = np.where(
    df_creditos_pagos['mes_apertura'] >= FECHA_CORTE,
    df_creditos_pagos['saldo_posterior_intereses_acumulados'],
    df_creditos_pagos['saldo_posterior_valor_futuro']
)
df_creditos = df_creditos_pagos.groupby('id_credito').last()


In [31]:
# ---------------------------
# Cálculos de paridades
# ---------------------------
fecha_final = pd.Timestamp('2024-12-30')
df_creditos['paridad_inicial'] = fc.paridad_inicial(df_creditos, df_creditos_pagos, fecha_final)
df_creditos['paridad_final'] = fc.paridad(df_creditos, df_creditos_pagos, fecha_final)
df_creditos['saldo_inicial'] = fc.saldo_inicial(df_creditos, df_creditos_pagos, fecha_final)
df_creditos = fc.considerar(df_creditos, fecha_final)

roll_over = df_creditos.pivot_table('saldo_inicial', 'paridad_inicial', 'paridad_final', 'sum', 0)
roll_over_normalizado = roll_over.div(roll_over.sum(axis=1), axis=0) * 100

display(roll_over_normalizado)

paridad_final,Al Corriente,PAR 1,PAR 30,PAR 90
paridad_inicial,,,,
Al Corriente,98.015576,1.907713,0.070642,0.006069
PAR 1,11.675817,62.410427,25.913756,0.000000
PAR 30,1.382961,1.961160,63.192014,33.463864
PAR 90,0.078877,0.092936,0.494332,99.333855
